# Measurement files UT Loading

Notebook to load Measurement files ut data into the databse

## Imports

In [1]:
import psycopg2
import numpy as np
#import a folder in the parent directory
import sys
sys.path.append('../')
import dbtools.dbtools as qrs
from pathlib import Path
import tifffile as tiff

## Connection

In [2]:
try:
    # Connect to the PostgreSQL database
    conn = qrs.connect()
    print("Connected to the database")

except (Exception, psycopg2.DatabaseError) as error:
    print(error)

Connected to the database


## File loading

Here we have to load the file of the measurement to be saved in the database.

The parent file path has to be explained in depth: Measurements include more than one sample, but then the files are cropped so each sample can be treated individually. Parent file represents the file from where the one being saved is extracted from.

If the file being saved is not extracted from any other, set paren_file_path to None

In [3]:
parent_file_path =Path(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\03_UT_data\Probetas JI\medidas\2025 metodology\4+5+7+8+10+11+12\JI_4_5_7_8_10_11_12_77mmdistance_11db_10mmrange_20mms_ascan492x190x448.tif')

file_path = Path(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\03_UT_data\Probetas JI\medidas\2025 metodology\4+5+7+8+10+11+12\JI_4_5_7_8_10_11_12_77mmdistance_11db_10mmrange_20mms_ascan492x190x448.tif')

file = tiff.imread(file_path)

# Measurement type data

We have to set the measurment type id, in this case is the ultrasound 2024 JI methodology so the id is 1.

In case you are not sure of the mesaurement type id, you can check it in the measurement_type table in the next cell

In [4]:
parent_table_name = 'measurementtypes'

parent_id_column = parent_table_name[:-1] + '_id'

parent_data = qrs.get_data(parent_table_name)

parent_data

,id_measurementtype,name_measurementtype,description_measurementtype
0,2,XCT_2024,"First XCT measurements, 2024 metodology."
1,3,UT_2024,"UT measurements, 2024 metodology. Used for the..."
2,4,UT_2025_1,"UT measurements, 2025 metodology. Measuring at..."


In [5]:
#we set the id
parent_id = 4

## Measurement data

Now we have to set the attributes and metadata of the measurements:

1. Main attributes

    1. Sample_names: Its a list with the names of all the samples present in the file. If the names are not written as in the database it wont work so be precise.

    2. file_path: The file location

    3. Parent_file_id: In this variable save the path to the parent file. A parent file is the file where the file we want to save was precomputed from, for example the amplitude file is computed from the rf file. This will be automatically extracted later.

2. Metadata

    1. Dimensions: Dimensions are automatically computed from the loaded file

    2. Dtype: The precision type of the file

    3. File_type: The file extension

    4. Signal_type: The signal type can be RF, amplitude, I/Q or whatever signal processing result
    

In [ ]:
sample_names = ['JI_4']

main_parameters = {'file_path':file_path,parent_id_column:parent_id}

# metadata

#for each parameter of the measurement a metadata has to be created

metadata_parameters = []

#dimensions
metadata_parameters.append({'key':'height', 'value':str(file.shape[0]), 'type':'cardinal'})

metadata_parameters.append({'key':'width', 'value':str(file.shape[1]), 'type':'cardinal'})

metadata_parameters.append({'key':'depth', 'value':str(file.shape[2]), 'type':'cardinal'})

#dtype

metadata_parameters.append({'key':'dtype', 'value':str(file.dtype), 'type':'nominal'})

#file type

metadata_parameters.append({'key':'file_type', 'value':file_path.suffix[1:], 'type':'nominal'})

#signal_type
metadata_parameters.append({'key':'signal_type', 'value':'RF', 'type':'nominal'})


## Parent file ID

A measurement file can be extracted from another one, for exmaple in UT we measure in the same inspection samples 1,2,3 and 4. An then we crop them individually from the initial file so we have a separate file for each of the samples.

This relationship let us keep track of this process.

However if the file is the initial one, there is no need to fill this paremeter

In [ ]:
if parent_file_path is not None:

    parent_file_id_table_name = 'measurements'

    parent_file_data = qrs.get_data_metadata(parent_file_id_table_name)

    parent_file_id = parent_file_data[parent_file_data['file_path_measurement'] == str(parent_file_path)]['id_measurement']

    main_parameters['parent_measurement_id'] = parent_file_id.values[0]

    print('Parent measurement id: ', parent_file_id.values[0])

## Load into the table

In [ ]:
print('Parameters to be inserted: ')
for key, value in main_parameters.items():
    print(f"-    {key}: {value}")

In [ ]:
table_name = 'measurements'

# Extract column names and values from the attributes dictionary
columns = ', '.join(main_parameters.keys())
values = ', '.join([f"'{v}'" for v in main_parameters.values()])

# Construct the SQL INSERT statement
sql = f"INSERT INTO {table_name} ({columns}) VALUES ({values})"

print(sql)

# Create a cursor object using the cursor() method
cursor = conn.cursor()

# Execute the SQL statement
cursor.execute(sql)

cursor.execute('COMMIT')

cursor.close()

## Make sure insert was correct

In [ ]:
data = qrs.data_parent(table_name,parent_table_name,parent_id_column)

data

Now we store the id of the inserted row, to insert its related metadata

In [ ]:
row_id = data['id_measurement'].values[-1]

print(f"Row ID: {row_id}")

## Load into the metadata table

In [ ]:
metadata_table_name =  table_name[:-1] + '_metadata'

for attributes in metadata_parameters:

    attributes[table_name[:-1] + '_id'] = row_id

    # Extract column names and values from the attributes dictionary
    columns = ', '.join(attributes.keys())
    values = ', '.join([f"'{v}'" for v in attributes.values()])

    # Construct the SQL INSERT statement
    sql = f"INSERT INTO {metadata_table_name} ({columns}) VALUES ({values})"

    print(sql)

    # Create a cursor object using the cursor() method
    cursor = conn.cursor()

    # Execute the SQL statement
    cursor.execute(sql)

    cursor.execute('COMMIT')

    cursor.close()

## Make sure insert was correct

In [ ]:
data = qrs.data_parent(table_name,parent_table_name,parent_id_column)

data

## Load in the sample measurement table

In a measurement from 1 to n samples can be present.

We save them in the sample_measurements table.

We have to locate the samples that are present in this measurement to load them in a table that relates them.

We need the ids of the samples so we filter the samples table to get the ids of the samples in the samples names list

In [ ]:
samples_data = qrs.get_data_metadata('samples')

#get the ids of the samples in sample_names
samples_data = samples_data[samples_data['name_sample'].isin(sample_names)]

sample_ids = samples_data['id_sample'].values.tolist()

print('sample_ids', sample_ids)

In [ ]:
relational_table_name = 'sample_measurements'

for sample_id in sample_ids:

    relational_parameters = {'sample_id': sample_id, 'measurement_id': row_id}

    # Extract column names and values from the attributes dictionary
    columns = ', '.join(relational_parameters.keys())
    values = ', '.join([f"'{v}'" for v in relational_parameters.values()])

    # Construct the SQL INSERT statement
    sql = f"INSERT INTO {relational_table_name} ({columns}) VALUES ({values})"

    print(sql)

    # Create a cursor object using the cursor() method
    cursor = conn.cursor()

    # Execute the SQL statement
    cursor.execute(sql)

    cursor.execute('COMMIT')

    cursor.close()

## Make sure insert was correct

In [ ]:
data = qrs.relation_metadata(table_name,'samples',relational_table_name)

data